In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import itertools

import torch
from torch import nn

import youtokentome as yttm

from utils.poetry import *
from utils.transformer_tools import *

In [2]:
def gen_some_sent(gen_model, phrase, N, beam_size, need_reweight=False, temperature=0, alpha=0):
    """
    Функция для генерации нескольких различных продолжений фразы с возможностью настройки параметров температуры и сглаживания меток
    """
    answers = []
    for i in range(N):
        beam_gen_variants = gen_model(phrase, beamsize=beam_size, return_hypotheses_n=1, need_reweight=need_reweight, temperature=temperature, alpha=alpha)
        for score, pred_txt in beam_gen_variants:
            answers.append(pred_txt)
    return answers

def test_gen(gen_model, text, temperatures, alphas):
    """
    Фукнция для перебора комбинаций температуры и сглаживания для генерации продолжения фразы
    """
    all_test = itertools.product(temperatures, alphas)
    for t, a in all_test:
        print(f"Температура = {t}, альфа = {a}")
        answers = gen_some_sent(gen_model, text, 5, 10, True, t, a)
        for ans in answers:
            print(ans)
        print("-----")

In [3]:
BPE_MODEL_FILENAME = './models/tokens/lord_bpe_1000_new.yttm'
tokenizer = yttm.BPE(BPE_MODEL_FILENAME)

G_model = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 256,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 256,
            nhead = 16,
            dim_feedforward = 512,
            dropout = 0.1
        ),
        num_layers=5
    ),
    emb_dropout = 0.1
)

In [6]:
temps = [0.01, 0.2, 0.5, 0.8]
alps = [0, 0.1, 0.3, 0.5]

### Модель после генеративно-состязательной сети

In [4]:
G_model.load_state_dict(torch.load('./models/GAN/4/model_G.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [8]:
answers = []
for i in range(5):
    beam_gen_variants = beam_generator("Today we are", beamsize=10, return_hypotheses_n=1, need_reweight=True, temperature=0.9, alpha=0.8)
    for score, pred_txt in beam_gen_variants:
        answers.append(pred_txt)

for ans in answers:
    print(ans)

Today we are weary, and weary and weariness.' 'Then we are going together,' said Treebeard. 'The Ents are Ents and Men, and Entwi
Today we are lost.' The hobbits looked at themselves and smiled. There was no answer. They were only a few prisoners, and they had not seen themselves in the
Today we are done.' 'Yes,' said Pippin. 'My dear hobbit, I don't know. I don't suppose I don't know what I can't believe. I don't
Today we are going to Mordor, and that is not far from the North, and the Nine of Minas Tirith.' `But if I do not wish to go,' said
Today we are drawing to the Ford.' 'Or secret ways!' said Gimli. 'Look!' said Legolas. 'Look!' said Legolas. 'Let us go!'


In [9]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0, альфа = 0
Today we are drawing near to the North-gate Middle-earth.' 'Then you must find the Company,' said Eomer. 'Look!' said Theo
Today we are drawing near to the Fords. But we are not going to the Moon. We must go to Minas Tirith.' `This is a saddle
Today we are drawing near to the Ford When the Ford of Bruinen the Brandywine River, and the Book of the Bucklebury
Today we are called _The Pony,_, _Pleas_,_ he said. _It_ drink_, _gollum_ – _gollum_, _gollum
Today we are. Tom Bombadil, Tombadil, Tom Bombadil, Bombadil, Bombadil, Bombadil
-----
Температура = 0, альфа = 0.1
Today we are going to the Ford of Isen. But we are not far from the Emyn Muil to the Misty Mountains of Middle-earth. We
Today we are going to the Fords of Bruinen, and that is not far away.' 'Then let us go on,' said Aragorn. 'Then let us go!' said Gimli. 'Then
Today we are summoned to the Entwash to the Entwash of the Entwives. There are Ents and Ents and Entmoots and Entwi
Today we are drawing nea

In [10]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0, альфа = 0
For the King and Elendil's son of Eorl, and the Third Marshal of Denethor, the Third Marshal of the Dark
For the King and Elendil's son of Arathorn, the Third Marshal of the Third Mark, the Third Marshal of the D
For the King and Elendil's son of Elendil's son of Eomundil, and the Third Marshal of Gondor, the Third
For the King and Elendil's son of Elendil's son of Elendil, and the Third Marshal of Elendil's son of D
For the King and the Dwarf, and the Dark Lord of the Mark Lordor, and the Shadow of the Mark Lord of the Mountains of Shadowfax
-----
Температура = 0, альфа = 0.1
For the King and the Lord of the Mark of Minas Tirith, the Ring of Minas Tirith.' `Nor Boromir son of Elendil,' said Aragorn,
For the King and Elendil's son of Elendil's son of Elendil's son of Elendil's son of Elendil's son of
For the King and the Greyfolden Haldir, and the Lady Galadriel of the Lady Galadriel who servants of Gondor
For the King and Elendil's son of Elendil's son of El

In [11]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0, альфа = 0
We are going to Mordor.' `But what are you going? ' asked Sam. `And what is it? ' `Yes, yes,' said Gollum. `Smeagol doesn't
We are going to Mordor. We must go to Minas Tirith.' `And what is the Elves? ' said Faramir. `Not Elves,' said Faramir. `
We are going tomorrow.' 'Yes, yes,' said Gollum. `No, no, no, no! ' said Gollum. `No, no! ' said Gollum. `No
We are going to Mordor.' `But we are not going to Minas Tirith,' said Faramir, `and I do not know what I do. But there is no doubt that we
We are going tomorrow.' 'It is a good deal,' said Gimli. 'But I have not heard of the Riders of Rohan. I have heard of the Riders of Rohan,
-----
Температура = 0, альфа = 0.1
We are going to Bucklebury.' 'My dear Brandybuck!' said Treebeard. 'I don't think you've come back to the Brandywine B
We are going to. And I'm sorry, I can't get a chance.' 'Yes, yes,' said Sam. 'But I'm sorry, I'm sorry
We are going to the Ferry.' 'Very well,' said Bilbo. 'But you don't want to know, 

In [12]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0, альфа = 0
That long story is called to the North, and the North West-galin's son of Elendil's son of Elendil's son of E
That long story is a song of the Enemy, and the Third Marshal of Minas Tirith, the Third Marshal of Mirkwood
That long story is known to me, and I have never heard of it before.' 'Oh yesterday!' said Gimli. 'Let me go!' said Legolas. 'This is the
That long story is dangerous. We are not far away, and we are not far away.' 'Yes,' said Gimli. 'Maybe,' said Legolas. 'Let us hope that we have
That long story is the son of Elendil, and the Third Marshal of the Dunedain, and the Third Marshal of the Dark Lord
-----
Температура = 0, альфа = 0.1
That long story is the oldest of the Enemyn Muil, and the Third Marshal Men of Minas Tirith lie in the Sea, and
That long story is destroyed in the North, and the Northfarthing lies between the Mountains of Middle-earth, and the Sea.
That long story is beyond the North, and the Third Marshal of Durin, the Dark Lord of

### Модель до генеративно-состязательной сети

In [17]:
G_model.load_state_dict(torch.load('./models/Lord/4/Lord.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [18]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0, альфа = 0
Today we are drawell to the Ford of Rivendell.' 'My dear friends!' said Theoden. 'Here is the Elf-lord of Elend
Today we are fought of the Mark of the Dunedain.' 'Then let us go!' said Gimli. 'I do not know what I mean?' 'I do not know,' answered Gimli. 'L
Today we are done.' 'Do not let us go!' said Gimli. 'Let us go, and go!' said Legolas. 'I do not know what you mean?' 'I do not know what I
Today we are drawing near to the South-gate of the Emyn Muil. That is the way of the Enemy's Dark Lord, and
Today we are going to Moria. We must go to Minas Tirith, if we go to warnorrow we will.' `And what do you wish to do? ' asked
-----
Температура = 0, альфа = 0.1
Today we are come to the Entwash, and there are Ents and Entwives. Ents are Ents and Ents, Entwives are Ents, and Ents, Ents, Ent
Today we are going to Mordor. We must go to the Fords of Moria.' `Ah! ' said Frodo. `But what do you mean to do? ' said Faramir
Today we are our friendship,' said Aragorn. 'Ther

In [19]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0, альфа = 0
For the King and the Dark Lord, and the Galadhrim of Rohan, and the Green Wormtongue's son of Arathorn, and
For the King and the White Mountains of the Mountains, and the Dark Lord of the Mountains of Minas Tirith.' `Nor Boromir! ' said Faramir
For the King and the Lady Galadriel,' he said. 'Farewell to the Lady Galadriel speak of the Lady of the Galadhrim, and
For the King and the Lady of the Lady Galadhrim, and the Galadhrim, and the Greyfladden Fields of the Gondor
For the King and Glorfindel, and the Dwarf, and the Dwarf-streams of the Dark Lord in the Mountains of Middle-ear
-----
Температура = 0, альфа = 0.1
For the King and the Lady of Rohan, and the Galadriel wields of the Galadhrim, and the Greyrim of the North.
For the King and the Wizards of the Dark Tower of Minas Tirith, and the Dark Lord of the Mountains of Middle-earth, and the
For the King and Brandybucks of Buckland, and the Ferry Brandywine Bolgers, and Bolgers, Bolgers, Bolger
For the King 

In [20]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0, альфа = 0
We are going tomorrow.' 'Or secret ways,' said Gimli. 'Let us go to Minas Tirith, and the Elves of Minas Tirith, and
We are going to Mordor. We must go to Minas Tirith.' `Then we must go together,' said Boromir. `But we must go, we shall go with him.' `You
We are going to Bucklebury the Brandywine Bridge.' 'Behold Bagginses, Bolgers, Bracegers, Baggin
We are going to Mordor. But if you've come back, I'll warrant.' 'Yes, yes,' said Sam, 'but I'm afraid. I'm
We are going to Bucklebury.' 'My dear old Brandybuck!' said Bilbo. 'Oh, I don't think you've come to Bucklebury
-----
Температура = 0, альфа = 0.1
We are going to stay here, if I can. I can't see anyway.' 'Oh yes, yes,' said Gollum. 'Yes, yes, yes, yes, yes, yes, y
We are going together.' 'Yes, yes, we are going,' said Pippin. 'My dear Bagginses.' 'Maybe,' said Merry. 'But I can't imagin
We are going to Mordor, if we're going to. And we've got a bit of a bit, we've got a bit of biting.' `No,
We are going to

In [21]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0, альфа = 0
That long story is in the West of Middle-earth.' 'Then she son of Eomer son of Arathorn,' she said to the Lady Galadriel
That long story is dangerous.' `But what is it? ' asked Frodo. `_The Precious? ' `No, no! ' said Gollum. `No, no! '
That long story is called, and the Third Marshal of Gondor, and the Third Marshal of the Dark Lord of Minas Tirith, and
That long story is in the Westfolden. I am not sure of the Enemy is in the West, and the West of the Westfold of the Westf
That long story is called Brandybucks of Bree-land, and the Bridge of the Brandywine Bridge of the Brandywine Bridge,
-----
Температура = 0, альфа = 0.1
That long story is according to the Bridge of Mordor. Indeed I am not sure. I am not going to do what I mean. I am going to do,
That long story is the son of Elendil's son of Elendil, and the Third Marshal of the Third Marshal of Minas T
That long story is called _The Pony,_, _When Smeagol_ and the Brandywine Brandywine Brandybucks – _gol